### S3로 데이터 적재

In [4]:
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="mlops/access/data_loader")

In [36]:
from pydatafabric.ye import get_spark
import os

#valut를 통해 aws credential 정보 로딩

os.environ["AWS_ACCESS_KEY_ID"] = aws_info["aws_access_key_id"]
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_info["aws_secret_access_key"]

spark = get_spark()

In [19]:
review_keywords = spark.read.format('bigquery') \
  .option('table', 'smart-ruler-304409:product_attribute_module.app_review_keywords') \
  .load()
review_keywords.createOrReplaceTempView('review_keywords')

In [20]:
query="select * from review_keywords"

In [21]:
df = spark.sql(query)
df.show(5)

+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+-------------------------------+
|      prdt_cd|store_cd|order_date|                            sku_name|           link_sku_name|                           comment|score_text|                       keywords|
+-------------+--------+----------+------------------------------------+------------------------+----------------------------------+----------+-------------------------------+
|1102150000000|    1018|  20220102|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|    기름기가 많아서 
먹기 불편했음|       Bad|    기름기 많아서 먹기 불편했음|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 기름이 너무너무 많아요.버린 기...|       Bad|       기름 많아요 기름 무게 만|
|1102150000000|    1108|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살|세일전주에 샀을때 손질해보니 기...|       Bad|전주 때 손질 기름 근육 고기 ...|
|1102150000000|    1048|  20220101|(후레쉬팩)미국냉장초이스척아이로스트|(후레쉬팩)미국산CH갈비살| 고기가 상태가 별로였어요
빨리 ...|      

In [22]:
env = ["dev"]

In [35]:
for k in env:
    s3_path = f"s3a://mlops-loader-dev/review_keywords"
    print(s3_path)

    aws_info = get_secrets(mount_point="datafabric", path=f"mlops/access/data_loader")
    spark.conf.set("fs.s3a.fast.upload.buffer", 'disk')   
    spark.conf.set("fs.s3a.buffer.dir", '/data/tmp')
    spark.conf.set("fs.s3a.fast.upload.buffer", 'bytebuffer')   
    spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
    spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

    spark.sql(query).write.mode("overwrite").parquet(s3_path)

s3a://mlops-loader-dev/review_keywords


22/04/19 22:07:47 WARN org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter: Exception get thrown in job commit, retry (1) time.
org.apache.hadoop.util.DiskChecker$DiskErrorException: Could not find any valid local directory for s3ablock-0001-
	at org.apache.hadoop.fs.LocalDirAllocator$AllocatorPerContext.getLocalPathForWrite(LocalDirAllocator.java:463)
	at org.apache.hadoop.fs.LocalDirAllocator$AllocatorPerContext.createTmpFileForWrite(LocalDirAllocator.java:477)
	at org.apache.hadoop.fs.LocalDirAllocator.createTmpFileForWrite(LocalDirAllocator.java:213)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.createTmpFileForWrite(S3AFileSystem.java:600)
	at org.apache.hadoop.fs.s3a.S3ADataBlocks$DiskBlockFactory.create(S3ADataBlocks.java:811)
	at org.apache.hadoop.fs.s3a.S3ABlockOutputStream.createBlockIfNeeded(S3ABlockOutputStream.java:190)
	at org.apache.hadoop.fs.s3a.S3ABlockOutputStream.<init>(S3ABlockOutputStream.java:168)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.create(S3AFileSystem

Py4JJavaError: An error occurred while calling o208.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apac

In [ ]:
spark.stop()